In [10]:
import pandas as pd
import numpy as np
import pyreadstat as st
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

In [11]:
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2024 3T\[MT06] Microeconometría I\Clases\Stata\Archivos Stata-20250107\laborsub.dta"

df, meta = st.read_dta(path)
df.head()

,lfp,whrs,kl6,k618,wa,we
0,1,1610,1,0,32,12
1,1,1656,0,2,30,12
2,1,1980,1,3,35,12
3,1,456,0,3,34,12
4,1,1568,1,2,31,14


In [12]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
import numpy as np

# Preparar los datos
X = sm.add_constant(df[["kl6", "k618", "wa", "we"]])  # Datos para las variables independientes
y = df["whrs"]  # Variable dependiente

# Definir función de verosimilitud
def log_likelihood(params, X, y):
    beta = params[:-1]
    sigma = np.abs(params[-1])  # Restricción de sigma > 0
    
    mu = X @ beta
    
    # Usar un epsilon pequeño para evitar logaritmos de cero
    epsilon = 1e-10
    pdf_values = norm.pdf(y, mu, sigma)
    cdf_values = norm.cdf(0, mu, sigma)
    
    # Asegurar que los valores no sean cero en el logaritmo
    log_pdf = np.log(np.maximum(pdf_values, epsilon))
    log_cdf = np.log(np.maximum(cdf_values, epsilon))
    
    ll = np.sum((y > 0) * log_pdf) + np.sum((y == 0) * log_cdf)
    
    return -np.sum(ll)  # Se minimiza la log-verosimilitud negativa

# Estimación MLE
init_params = np.ones(X.shape[1] + 1)  # Inicialización de parámetros
res = minimize(log_likelihood, init_params, args=(X, y), method="L-BFGS-B")
params = res.x

# Verificar la cantidad de parámetros y ajustarlos
param_names = ["Intercept"] + list(df[["kl6", "k618", "wa", "we"]].columns) + ["sigma"]

# Comprobar la longitud de ambos
print("Número de parámetros estimados:", len(params))
print("Número de nombres de parámetros:", len(param_names))

# Si el número de parámetros es correcto, crear el DataFrame de resultados
if len(params) == len(param_names):
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)
else:
    # Si hay un desajuste, ajustamos la cantidad de parámetros
    print(f"Desajuste en la longitud de parámetros: {len(params)} vs {len(param_names)}.")
    if len(params) > len(param_names):
        param_names += ["extra_param_" + str(i) for i in range(len(param_names), len(params))]
    elif len(params) < len(param_names):
        param_names = param_names[:len(params)]  # Recortamos si es necesario
    
    # Crear DataFrame ajustado
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)


Número de parámetros estimados: 6
Número de nombres de parámetros: 6
   Parameter     Estimate
0  Intercept   591.599993
1        kl6  -828.418990
2       k618  -140.121376
3         wa   -25.018650
4         we   103.628917
5      sigma -1310.119449


In [ ]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
import numpy as np

# Preparar los datos
df2 = df[df['whrs'] > 0]  # Filtrar solo observaciones no censuradas
X = sm.add_constant(df2[["kl6", "k618", "wa", "we"]])  # Datos para las variables independientes
y = df2["whrs"]  # Variable dependiente

# Definir función de verosimilitud con censura en 0
def log_likelihood(params, X, y):
    beta = params[:-1]
    sigma = np.abs(params[-1])  # Restricción de sigma > 0
    
    mu = X @ beta
    
    # Usar un epsilon pequeño para evitar logaritmos de cero
    epsilon = 1e-100
    pdf_values = norm.pdf(y, mu, sigma)
    cdf_values = norm.cdf(0, mu, sigma)
    
    # Log-verosimilitud para observaciones no censuradas (y > 0)
    log_pdf = np.log(np.maximum(pdf_values, epsilon))
    
    # Log-verosimilitud para observaciones censuradas en 0 (y == 0)
    log_cdf = np.log(np.maximum(cdf_values, epsilon))
    
    # La log-verosimilitud total
    ll = log_pdf - log_cdf
    
    return -np.sum(ll)  # Se minimiza la log-verosimilitud negativa

# Estimación MLE
init_params = np.ones(X.shape[1] + 1)  # Inicialización de parámetros
res = minimize(log_likelihood, init_params, args=(X, y), method="L-BFGS-B")
params = res.x

# Verificar la cantidad de parámetros y ajustarlos
param_names = ["Intercept"] + list(df[["kl6", "k618", "wa", "we"]].columns) + ["sigma"]

# Comprobar la longitud de ambos
print("Número de parámetros estimados:", len(params))
print("Número de nombres de parámetros:", len(param_names))

# Si el número de parámetros es correcto, crear el DataFrame de resultados
if len(params) == len(param_names):
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)
else:
    # Si hay un desajuste, ajustamos la cantidad de parámetros
    print(f"Desajuste en la longitud de parámetros: {len(params)} vs {len(param_names)}.")
    if len(params) > len(param_names):
        param_names += ["extra_param_" + str(i) for i in range(len(param_names), len(params))]
    elif len(params) < len(param_names):
        param_names = param_names[:len(params)]  # Recortamos si es necesario
    
    # Crear DataFrame ajustado
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)

Número de parámetros estimados: 6
Número de nombres de parámetros: 6
   Parameter     Estimate
0  Intercept   244.967224
1        kl6  -258.603046
2       k618   -63.641982
3         wa    -2.871990
4         we     5.887783
5      sigma -1528.873225


In [32]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
import numpy as np

# Preparar los datos
X = sm.add_constant(df[["kl6", "k618", "wa", "we"]])  # Datos para las variables independientes
y = df["whrs"]  # Variable dependiente

# Definir función de verosimilitud
def log_likelihood(params, X, y):
    beta = params[:-1]
    sigma = np.abs(params[-1])  # Restricción de sigma > 0
    
    mu = X @ beta
    
    # Usar un epsilon pequeño para evitar logaritmos de cero
    epsilon = 1e-100
    # pdf_values = norm.pdf(y, mu, sigma)
    # cdf_values = norm.cdf(0, mu, sigma)
    
    # Asegurar que los valores no sean cero en el logaritmo
    log_pdf = -0.5 * np.log(2 * np.pi) - 0.5 * np.log(sigma**2) - ((y - mu) ** 2) / (2 * sigma**2)
    log_cdf = np.log(np.maximum(1 - norm.cdf(mu / sigma), epsilon))
    
    ll = np.sum((y > 0) * log_pdf) + np.sum((y == 0) * log_cdf)
    
    return -np.sum(ll)  # Se minimiza la log-verosimilitud negativa

# Estimación MLE
init_params = np.ones(X.shape[1] + 1)  # Inicialización de parámetros
res = minimize(log_likelihood, init_params, args=(X, y), method="L-BFGS-B")
params = res.x

# Verificar la cantidad de parámetros y ajustarlos
param_names = ["Intercept"] + list(df[["kl6", "k618", "wa", "we"]].columns) + ["sigma"]

# Comprobar la longitud de ambos
print("Número de parámetros estimados:", len(params))
print("Número de nombres de parámetros:", len(param_names))

# Si el número de parámetros es correcto, crear el DataFrame de resultados
if len(params) == len(param_names):
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)
else:
    # Si hay un desajuste, ajustamos la cantidad de parámetros
    print(f"Desajuste en la longitud de parámetros: {len(params)} vs {len(param_names)}.")
    if len(params) > len(param_names):
        param_names += ["extra_param_" + str(i) for i in range(len(param_names), len(params))]
    elif len(params) < len(param_names):
        param_names = param_names[:len(params)]  # Recortamos si es necesario
    
    # Crear DataFrame ajustado
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)

Número de parámetros estimados: 6
Número de nombres de parámetros: 6
   Parameter     Estimate
0  Intercept   587.433584
1        kl6  -827.538027
2       k618  -139.951544
3         wa   -24.964737
4         we   103.738070
5      sigma  1309.832381


In [44]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
import numpy as np

# Preparar los datos
X = sm.add_constant(df[["kl6", "k618", "wa", "we"]])  # Datos para las variables independientes
y = df["whrs"]  # Variable dependiente

# Definir función de verosimilitud
def log_likelihood(params, X, y):
    beta = params[:-1]
    sigma = np.abs(params[-1])  # Restricción de sigma > 0
    
    mu = X @ beta
    
    # Usar un epsilon pequeño para evitar logaritmos de cero
    epsilon = 1e-100
    # pdf_values = norm.pdf(y, mu, sigma)
    # cdf_values = norm.cdf(0, mu, sigma)
    
    # Asegurar que los valores no sean cero en el logaritmo
    log_pdf = -0.5 * np.log(2 * np.pi) - 0.5 * np.log(sigma**2) - ((y - mu) ** 2) / (2 * sigma**2)
    log_cdf = np.log(np.maximum(1 - norm.cdf(mu / sigma), epsilon))
    
    ll = log_pdf + log_cdf
    
    return -np.sum(ll)  # Se minimiza la log-verosimilitud negativa

# Estimación MLE
init_params = np.ones(X.shape[1] + 1)  # Inicialización de parámetros
res = minimize(log_likelihood, init_params, args=(X, y), method="L-BFGS-B")
params = res.x

# Verificar la cantidad de parámetros y ajustarlos
param_names = ["Intercept"] + list(df[["kl6", "k618", "wa", "we"]].columns) + ["sigma"]

# Comprobar la longitud de ambos
print("Número de parámetros estimados:", len(params))
print("Número de nombres de parámetros:", len(param_names))

# Si el número de parámetros es correcto, crear el DataFrame de resultados
if len(params) == len(param_names):
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)
else:
    # Si hay un desajuste, ajustamos la cantidad de parámetros
    print(f"Desajuste en la longitud de parámetros: {len(params)} vs {len(param_names)}.")
    if len(params) > len(param_names):
        param_names += ["extra_param_" + str(i) for i in range(len(param_names), len(params))]
    elif len(params) < len(param_names):
        param_names = param_names[:len(params)]  # Recortamos si es necesario
    
    # Crear DataFrame ajustado
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)

Número de parámetros estimados: 6
Número de nombres de parámetros: 6
   Parameter     Estimate
0  Intercept    -5.351723
1        kl6  -292.050044
2       k618   -57.848851
3         wa    -8.431854
4         we    32.351200
5      sigma  1235.111405


In [62]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
import numpy as np

# Preparar los datos
df2 = df[df['whrs'] > 0]  # Filtrar solo observaciones no censuradas
X = sm.add_constant(df2[["kl6", "k618", "wa", "we"]])  # Variables independientes
y = df2["whrs"]  # Variable dependiente

# Definir función de log-verosimilitud para regresión truncada
def log_likelihood(params, X, y):
    beta = params[:-1]  # Coeficientes de regresión
    sigma = np.abs(params[-1])  # Restricción de sigma > 0
    
    mu = X @ beta  # Media condicional
    
    # Densidad normal en los valores observados
    log_pdf = -0.5 * np.log(2 * np.pi) - 0.5 * np.log(sigma**2) - ((y - mu) ** 2) / (2 * sigma**2)
    
    # CDF evaluada en el punto de truncamiento (0)
    log_cdf = np.log(np.maximum(1 - norm.cdf(0, loc=mu, scale=sigma), 1e-100))
    
    # Log-verosimilitud total
    ll = log_pdf - log_cdf
    
    return -np.sum(ll)  # Se minimiza la log-verosimilitud negativa

# Estimación MLE
init_params = np.ones(X.shape[1] + 1)  # Inicialización de parámetros
res = minimize(log_likelihood, init_params, args=(X, y), method="L-BFGS-B")

# Extraer los parámetros estimados
params = res.x
param_names = ["Intercept"] + list(df[["kl6", "k618", "wa", "we"]].columns) + ["sigma"]

# Crear DataFrame con resultados
results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
print(results_df)
print(res.message)  # Mensaje del optimizador
print(res.success)  # Indica si la optimización fue exitosa
print(res.nit)      # Número de iteraciones realizadas
print(res.fun)      # Valor final de la función objetivo

   Parameter     Estimate
0  Intercept  1584.614114
1        kl6  -802.626576
2       k618  -172.951631
3         wa    -8.781402
4         we    16.496883
5      sigma   984.141866
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
True
81
1200.915753776213


In [68]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
import numpy as np

# Preparar los datos
df2 = df[df['whrs'] > 0]  # Filtrar solo observaciones no censuradas
X = sm.add_constant(df2[["kl6", "k618", "wa", "we"]])  # Variables independientes
y = df2["whrs"]  # Variable dependiente

# Normalización de X para mejorar estabilidad numérica
X_mean = X.mean()
X_std = X.std()
X_std = X_std.replace(0, 1)  # Evitar divisiones por cero en columnas constantes
X = (X - X_mean) / X_std

# Definir función de log-verosimilitud para regresión truncada
def log_likelihood(params, X, y):
    beta = params[:-1]  # Coeficientes de regresión
    sigma = np.abs(params[-1])  # Restricción de sigma > 0
    
    mu = X @ beta  # Media condicional
    
    # Densidad normal en los valores observados
    log_pdf = -0.5 * np.log(2 * np.pi) - np.log(sigma) - ((y - mu) ** 2) / (2 * sigma**2)
    
    # CDF evaluada en el punto de truncamiento (0)
    log_cdf = np.log(np.maximum(1 - norm.cdf(0, loc=mu, scale=sigma), 1e-100))
    
    # Log-verosimilitud total
    ll = log_pdf - log_cdf
    
    return -np.sum(ll)  # Se minimiza la log-verosimilitud negativa

# Función de gradiente (primeras derivadas)
def gradient(params, X, y):
    beta = params[:-1]
    sigma = np.abs(params[-1])
    
    mu = X @ beta
    z = (y - mu) / sigma
    phi_z = norm.pdf(z)
    Phi_z = np.maximum(1 - norm.cdf(0, loc=mu, scale=sigma), 1e-100)

    # Gradiente respecto a beta
    grad_beta = X.T @ ((y - mu) / sigma**2 + phi_z / (sigma * Phi_z))
    
    # Gradiente respecto a sigma (corrigiendo posible inestabilidad numérica)
    grad_sigma = np.sum(((y - mu)**2 / sigma**3 - 1 / sigma) + phi_z * (y - mu) / (sigma**2 * Phi_z))
    
    return -np.append(grad_beta, grad_sigma)  # Minimizar, por eso el signo negativo

# Inicialización con OLS para mejor convergencia
ols_params = np.append(np.linalg.lstsq(X, y, rcond=None)[0], np.std(y))

# Estimación MLE usando trust-ncg (más estable que Newton-CG)
res = minimize(log_likelihood, ols_params, args=(X, y), method="Newton-CG", jac=gradient)

# Extraer los parámetros estimados
params = res.x

# Transformar coeficientes a la escala original de las variables
print(f"Longitud de params: {len(params)}")
# Transformar coeficientes a la escala original de las variables
params[1:-1] = params[1:-1] / X_std.values[1:]  # Desnormalizar coeficientes excepto el intercepto
params[0] -= np.sum(params[1:-1] * X_mean.values[1:] / X_std.values[1:])  # Ajuste del intercepto

# Nombres de los parámetros
param_names = ["Intercept"] + list(df[["kl6", "k618", "wa", "we"]].columns) + ["sigma"]

# Crear DataFrame con resultados
results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
print(results_df)

# Imprimir información de la optimización
print(f"Convergencia: {res.success}")
print(f"Iteraciones: {res.nit}")
print(f"Valor de la función de verosimilitud: {res.fun}")

Longitud de params: 6
   Parameter     Estimate
0  Intercept   187.768790
1        kl6  -322.206536
2       k618   -83.909154
3         wa    -7.104857
4         we     9.318657
5      sigma  1633.607870
Convergencia: False
Iteraciones: 3
Valor de la función de verosimilitud: 1211.0044012131093


In [ ]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
import numpy as np

# Preparar los datos
X = sm.add_constant(df[["kl6", "k618", "wa", "we"]])  # Datos para las variables independientes
y = df["whrs"]  # Variable dependiente

# Definir función de verosimilitud con censura en 0
def log_likelihood(params, X, y):
    beta = params[:-1]
    sigma = np.abs(params[-1])  # Restricción de sigma > 0
    
    mu = X @ beta
    
    # Usar un epsilon pequeño para evitar logaritmos de cero
    epsilon = 1e-100
    pdf_values = norm.pdf(y, mu, sigma)
    cdf_values = norm.cdf(0, mu, sigma)
    
    # Log-verosimilitud para observaciones no censuradas (y > 0)
    log_pdf = np.log(np.maximum(pdf_values, epsilon))
    
    # Log-verosimilitud para observaciones censuradas en 0 (y == 0)
    log_cdf = np.log(np.maximum(cdf_values, epsilon))
    
    # La log-verosimilitud total
    ll = np.sum((y > 0) * log_pdf) + np.sum((y == 0) * log_cdf)
    
    return -np.sum(ll)  # Se minimiza la log-verosimilitud negativa

# Estimación MLE
init_params = np.ones(X.shape[1] + 1)  # Inicialización de parámetros
res = minimize(log_likelihood, init_params, args=(X, y), method="L-BFGS-B")
params = res.x

# Verificar la cantidad de parámetros y ajustarlos
param_names = ["Intercept"] + list(df[["kl6", "k618", "wa", "we"]].columns) + ["sigma"]

# Comprobar la longitud de ambos
print("Número de parámetros estimados:", len(params))
print("Número de nombres de parámetros:", len(param_names))

# Si el número de parámetros es correcto, crear el DataFrame de resultados
if len(params) == len(param_names):
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)
else:
    # Si hay un desajuste, ajustamos la cantidad de parámetros
    print(f"Desajuste en la longitud de parámetros: {len(params)} vs {len(param_names)}.")
    if len(params) > len(param_names):
        param_names += ["extra_param_" + str(i) for i in range(len(param_names), len(params))]
    elif len(params) < len(param_names):
        param_names = param_names[:len(params)]  # Recortamos si es necesario
    
    # Crear DataFrame ajustado
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)

In [20]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
import numpy as np

# Preparar los datos
X = sm.add_constant(df[["kl6", "k618", "wa", "we"]])  # Datos para las variables independientes
y = df["whrs"]  # Variable dependiente

# Definir función de verosimilitud con truncamiento en 0
def log_likelihood_truncation(params, X, y):
    beta = params[:-1]
    sigma = np.abs(params[-1])  # Restricción de sigma > 0
    
    mu = X @ beta
    
    # Usar un epsilon pequeño para evitar logaritmos de cero
    epsilon = 1e-100
    pdf_values = norm.pdf(y, mu, sigma)
    cdf_values = norm.cdf(0, mu, sigma)
    
    # Log-verosimilitud para observaciones no truncadas (y > 0)
    log_pdf = np.log(np.maximum(pdf_values, epsilon))
    
    # Log-verosimilitud para observaciones truncadas en 0 (y <= 0)
    log_cdf = np.log(np.maximum(cdf_values, epsilon))
    
    # La log-verosimilitud total considerando truncamiento
    ll = np.sum((y > 0) * log_pdf) + np.sum((y <= 0) * log_cdf)
    
    return -np.sum(ll)  # Se minimiza la log-verosimilitud negativa

# Estimación MLE con truncamiento
init_params = np.ones(X.shape[1] + 1)  # Inicialización de parámetros
res = minimize(log_likelihood_truncation, init_params, args=(X, y), method="L-BFGS-B")
params = res.x

# Verificar la cantidad de parámetros y ajustarlos
param_names = ["Intercept"] + list(df[["kl6", "k618", "wa", "we"]].columns) + ["sigma"]

# Comprobar la longitud de ambos
print("Número de parámetros estimados:", len(params))
print("Número de nombres de parámetros:", len(param_names))

# Si el número de parámetros es correcto, crear el DataFrame de resultados
if len(params) == len(param_names):
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)
else:
    # Si hay un desajuste, ajustamos la cantidad de parámetros
    print(f"Desajuste en la longitud de parámetros: {len(params)} vs {len(param_names)}.")
    if len(params) > len(param_names):
        param_names += ["extra_param_" + str(i) for i in range(len(param_names), len(params))]
    elif len(params) < len(param_names):
        param_names = param_names[:len(params)]  # Recortamos si es necesario
    
    # Crear DataFrame ajustado
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)

Número de parámetros estimados: 6
Número de nombres de parámetros: 6
   Parameter     Estimate
0  Intercept   590.431073
1        kl6  -827.875466
2       k618  -140.078680
3         wa   -24.994961
4         we   103.635018
5      sigma -1309.961567


In [19]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize
import numpy as np

# Verificar dimensiones de X y y
print("Dimensiones de X:", X.shape)
print("Dimensiones de y:", y.shape)

# Asegurarse de que X tenga forma (n_samples, n_features)
X = sm.add_constant(df[["kl6", "k618", "wa", "we"]])  # Datos para las variables independientes
y = df["whrs"]  # Variable dependiente

# Asegurarse de que y esté como un vector de una dimensión
y = y.values  # Convertir a un arreglo NumPy si es un DataFrame de pandas

# Definir función de verosimilitud con censura en 0
def log_likelihood(params, X, y):
    beta = params[:-1]
    sigma = np.abs(params[-1])  # Restricción de sigma > 0
    
    mu = X @ beta
    
    # Usar un epsilon pequeño para evitar logaritmos de cero
    epsilon = 1e-100
    pdf_values = norm.pdf(y, mu, sigma)
    cdf_values = norm.cdf(0, mu, sigma)
    
    # Log-verosimilitud para observaciones no censuradas (y > 0)
    log_pdf = np.log(np.maximum(pdf_values, epsilon))
    
    # Log-verosimilitud para observaciones censuradas en 0 (y == 0)
    log_cdf = np.log(np.maximum(cdf_values, epsilon))
    
    # La log-verosimilitud total
    ll = np.sum((y > 0) * log_pdf) + np.sum((y == 0) * log_cdf)
    
    return -np.sum(ll)  # Se minimiza la log-verosimilitud negativa

# Definir el jacobiano de la log-verosimilitud
def log_likelihood_jacobian(params, X, y):
    beta = params[:-1]
    sigma = np.abs(params[-1])  # Restricción de sigma > 0
    
    mu = X @ beta
    
    pdf_values = norm.pdf(y, mu, sigma)
    cdf_values = norm.cdf(0, mu, sigma)
    
    # Gradiente con respecto a los parámetros beta
    d_ll_dbeta = np.sum((y > 0) * (y - mu) / sigma**2 * X, axis=0)
    
    # Gradiente con respecto a sigma
    d_ll_dsigma = np.sum((y > 0) * (mu - y)**2 * pdf_values / (sigma**3) - pdf_values / sigma)
    
    # Concatenar el gradiente
    grad = np.concatenate([d_ll_dbeta, [d_ll_dsigma]])
    
    return -grad  # Se minimiza, por lo que el gradiente debe ser negativo

# Definir el hessiano de la log-verosimilitud
def log_likelihood_hessian(params, X, y):
    beta = params[:-1]
    sigma = np.abs(params[-1])  # Restricción de sigma > 0
    
    mu = X @ beta
    
    pdf_values = norm.pdf(y, mu, sigma)
    cdf_values = norm.cdf(0, mu, sigma)
    
    # Inicializamos la matriz Hessiana de tamaño (n_params, n_params), donde n_params = len(beta) + 1
    n_params = X.shape[1] + 1  # Número de parámetros (coeficientes beta + sigma)
    hess = np.zeros((n_params, n_params))
    
    # Segunda derivada con respecto a los parámetros beta (parte de la Hessiana de beta)
    for i in range(X.shape[1]):
        for j in range(X.shape[1]):
            hess[i, j] = np.sum((y > 0) * (X[:, i] * X[:, j]) / sigma**2 * pdf_values)

    # Segunda derivada con respecto a sigma (parte de la Hessiana de sigma)
    for i in range(X.shape[1]):
        hess[i, -1] = np.sum((y > 0) * (X[:, i] * (mu - y)) / sigma**3 * pdf_values)
        hess[-1, i] = hess[i, -1]  # La Hessiana es simétrica
    
    # Segunda derivada con respecto a sigma (d2_ll_dsigma2)
    hess[-1, -1] = np.sum((y > 0) * (mu - y)**2 * pdf_values / (sigma**3)) - np.sum(pdf_values / sigma)
    
    return hess

# Estimación MLE utilizando el método trust-ncg (Newton-CG)
init_params = np.ones(X.shape[1] + 1)  # Inicialización de parámetros
res = minimize(log_likelihood, init_params, args=(X, y), method="trust-ncg", jac=log_likelihood_jacobian, hess=log_likelihood_hessian)
params = res.x

# Verificar la cantidad de parámetros y ajustarlos
param_names = ["Intercept"] + list(df[["kl6", "k618", "wa", "we"]].columns) + ["sigma"]

# Comprobar la longitud de ambos
print("Número de parámetros estimados:", len(params))
print("Número de nombres de parámetros:", len(param_names))

# Si el número de parámetros es correcto, crear el DataFrame de resultados
if len(params) == len(param_names):
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)
else:
    # Si hay un desajuste, ajustamos la cantidad de parámetros
    print(f"Desajuste en la longitud de parámetros: {len(params)} vs {len(param_names)}.")
    if len(params) > len(param_names):
        param_names += ["extra_param_" + str(i) for i in range(len(param_names), len(params))]  
    elif len(params) < len(param_names):
        param_names = param_names[:len(params)]  # Recortamos si es necesario
    
    # Crear DataFrame ajustado
    results_df = pd.DataFrame({"Parameter": param_names, "Estimate": params})
    print(results_df)


Dimensiones de X: (250, 5)
Dimensiones de y: (250,)


InvalidIndexError: (slice(None, None, None), 0)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   lfp     250 non-null    int64
 1   whrs    250 non-null    int64
 2   kl6     250 non-null    int64
 3   k618    250 non-null    int64
 4   wa      250 non-null    int64
 5   we      250 non-null    int64
dtypes: int64(6)
memory usage: 11.8 KB


In [28]:
meta.column_labels

['1 if woman worked in 1975',
 "Wife's hours of work",
 '# of children younger than 6',
 '# of children between 6 and 18',
 "Wife's age",
 "Wife's educational attainment"]

In [29]:
df['lfp'].value_counts()

lfp
1    150
0    100
Name: count, dtype: int64

In [30]:
df[['whrs']].describe().T

,count,mean,std,min,25%,50%,75%,max
whrs,250.0,799.84,915.60348,0.0,0.0,406.5,1599.75,4950.0


In [31]:
#¿Qué sucede si queremos estimar una función de oferta de trabajo?
import statsmodels.api as sm

# Filtrar datos donde whrs > 0
df_filtered = df[df["whrs"] > 0]

# Definir variables dependiente (Y) e independientes (X)
y = df_filtered["whrs"]
X = df_filtered[["kl6", "k618", "wa", "we"]]

# Agregar constante para la intersección (equivalente a _cons en Stata)
X = sm.add_constant(X)

# Ajustar el modelo
model = sm.OLS(y, X).fit()

# Mostrar resultados
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   whrs   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     2.802
Date:                Sun, 23 Feb 2025   Prob (F-statistic):             0.0281
Time:                        00:54:19   Log-Likelihood:                -1214.6
No. Observations:                 150   AIC:                             2439.
Df Residuals:                     145   BIC:                             2454.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1629.8168    615.130      2.650      0.0

In [32]:
from statsmodels.discrete.count_model import Tobit

# Definir variables dependiente (Y) e independientes (X)
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]

# Agregar constante para la intersección (equivalente a _cons en Stata)
X = sm.add_constant(X)

# Ajustar el modelo Tobit con censura inferior en 0
model = Tobit(y, X).fit(censor_lower=0)

# Mostrar resultados
print(model.summary())

ImportError: cannot import name 'Tobit' from 'statsmodels.discrete.count_model' (c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\discrete\count_model.py)

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import norm
from scipy.optimize import minimize

# Definir variables dependiente (Y) e independientes (X)
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]

# Agregar constante
X = sm.add_constant(X)

# Función de log-verosimilitud del modelo Tobit (censura inferior en 0)
def tobit_loglik(params, y, X):
    beta = params[:-1]  # Coeficientes de regresión
    sigma = params[-1]  # Desviación estándar del error
    
    if sigma <= 0:
        return np.inf  # Evitar valores no válidos
    
    xb = X @ beta  # Producto matricial X * beta
    uncensored = (y > 0)
    
    # Parte no censurada: Log-verosimilitud de la normal
    ll_uncensored = -0.5 * np.log(2 * np.pi) - np.log(sigma) - (y - xb) ** 2 / (2 * sigma**2)
    
    # Parte censurada: Probabilidad acumulada de la normal (usando scipy.stats.norm)
    ll_censored = np.log(np.maximum(norm.cdf(xb / sigma), 1e-10))  # Evita log(0)
    
    # Combinar log-verosimilitudes
    loglik = np.where(uncensored, ll_uncensored, ll_censored)
    
    return -np.sum(loglik)  # Maximizar la verosimilitud minimizando la función de pérdida

# Estimación de parámetros iniciales
init_params = np.append(np.zeros(X.shape[1]), 1)  # Coeficientes iniciales + sigma

# Optimización numérica
res = minimize(tobit_loglik, init_params, args=(y, X), method="BFGS")

# Resultados
beta_est = res.x[:-1]  # Coeficientes estimados
sigma_est = res.x[-1]  # Estimación de sigma

print("Coeficientes estimados:", beta_est)
print("Sigma estimado:", sigma_est)



c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_numdiff.py:592: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


Coeficientes estimados: [-5200401.23618509  1717029.03177521   313985.12206049    94771.03642374
    44524.92114298]
Sigma estimado: 510055.98735519126


In [ ]:
import math
import warnings

import numpy as np
import pandas as pd
from scipy.optimize import minimize
import scipy.stats
from scipy.special import log_ndtr
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error


def split_left_right_censored(x, y, cens):
    counts = cens.value_counts()
    if -1 not in counts and 1 not in counts:
        warnings.warn("No censored observations; use regression methods for uncensored data")
    xs = []
    ys = []

    for value in [-1, 0, 1]:
        if value in counts:
            split = cens == value
            y_split = np.squeeze(y[split].values)
            x_split = x[split].values

        else:
            y_split, x_split = None, None
        xs.append(x_split)
        ys.append(y_split)
    return xs, ys


def tobit_neg_log_likelihood(xs, ys, params):
    x_left, x_mid, x_right = xs
    y_left, y_mid, y_right = ys

    b = params[:-1]
    # s = math.exp(params[-1])
    s = params[-1]

    to_cat = []

    cens = False
    if y_left is not None:
        cens = True
        left = (y_left - np.dot(x_left, b))
        to_cat.append(left)
    if y_right is not None:
        cens = True
        right = (np.dot(x_right, b) - y_right)
        to_cat.append(right)
    if cens:
        concat_stats = np.concatenate(to_cat, axis=0) / s
        log_cum_norm = scipy.stats.norm.logcdf(concat_stats)  # log_ndtr(concat_stats)
        cens_sum = log_cum_norm.sum()
    else:
        cens_sum = 0

    if y_mid is not None:
        mid_stats = (y_mid - np.dot(x_mid, b)) / s
        mid = scipy.stats.norm.logpdf(mid_stats) - math.log(max(np.finfo('float').resolution, s))
        mid_sum = mid.sum()
    else:
        mid_sum = 0

    loglik = cens_sum + mid_sum

    return - loglik


def tobit_neg_log_likelihood_der(xs, ys, params):
    x_left, x_mid, x_right = xs
    y_left, y_mid, y_right = ys

    b = params[:-1]
    # s = math.exp(params[-1]) # in censReg, not using chain rule as below; they optimize in terms of log(s)
    s = params[-1]

    beta_jac = np.zeros(len(b))
    sigma_jac = 0

    if y_left is not None:
        left_stats = (y_left - np.dot(x_left, b)) / s
        l_pdf = scipy.stats.norm.logpdf(left_stats)
        l_cdf = log_ndtr(left_stats)
        left_frac = np.exp(l_pdf - l_cdf)
        beta_left = np.dot(left_frac, x_left / s)
        beta_jac -= beta_left

        left_sigma = np.dot(left_frac, left_stats)
        sigma_jac -= left_sigma

    if y_right is not None:
        right_stats = (np.dot(x_right, b) - y_right) / s
        r_pdf = scipy.stats.norm.logpdf(right_stats)
        r_cdf = log_ndtr(right_stats)
        right_frac = np.exp(r_pdf - r_cdf)
        beta_right = np.dot(right_frac, x_right / s)
        beta_jac += beta_right

        right_sigma = np.dot(right_frac, right_stats)
        sigma_jac -= right_sigma

    if y_mid is not None:
        mid_stats = (y_mid - np.dot(x_mid, b)) / s
        beta_mid = np.dot(mid_stats, x_mid / s)
        beta_jac += beta_mid

        mid_sigma = (np.square(mid_stats) - 1).sum()
        sigma_jac += mid_sigma

    combo_jac = np.append(beta_jac, sigma_jac / s)  # by chain rule, since the expression above is dloglik/dlogsigma

    return -combo_jac


class TobitModel:
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
        self.ols_coef_ = None
        self.ols_intercept = None
        self.coef_ = None
        self.intercept_ = None
        self.sigma_ = None

    def fit(self, x, y, cens, verbose=False):
        """
        Fit a maximum-likelihood Tobit regression
        :param x: Pandas DataFrame (n_samples, n_features): Data
        :param y: Pandas Series (n_samples,): Target
        :param cens: Pandas Series (n_samples,): -1 indicates left-censored samples, 0 for uncensored, 1 for right-censored
        :param verbose: boolean, show info from minimization
        :return:
        """
        x_copy = x.copy()
        if self.fit_intercept:
            x_copy.insert(0, 'intercept', 1.0)
        else:
            x_copy.scale(with_mean=True, with_std=False, copy=False)
        init_reg = LinearRegression(fit_intercept=False).fit(x_copy, y)
        b0 = init_reg.coef_
        y_pred = init_reg.predict(x_copy)
        resid = y - y_pred
        resid_var = np.var(resid)
        s0 = np.sqrt(resid_var)
        params0 = np.append(b0, s0)
        xs, ys = split_left_right_censored(x_copy, y, cens)

        result = minimize(lambda params: tobit_neg_log_likelihood(xs, ys, params), params0, method='BFGS',
                          jac=lambda params: tobit_neg_log_likelihood_der(xs, ys, params), options={'disp': verbose})
        if verbose:
            print(result)
        self.ols_coef_ = b0[1:]
        self.ols_intercept = b0[0]
        if self.fit_intercept:
            self.intercept_ = result.x[1]
            self.coef_ = result.x[1:-1]
        else:
            self.coef_ = result.x[:-1]
            self.intercept_ = 0
        self.sigma_ = result.x[-1]
        return self

    def predict(self, x):
        return self.intercept_ + np.dot(x, self.coef_)

    def score(self, x, y, scoring_function=mean_absolute_error):
        y_pred = np.dot(x, self.coef_)
        return scoring_function(y, y_pred)

In [ ]:
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]
X = sm.add_constant(X)
df['cens'] = np.where(df['whrs'] == 0, -1, 0)
cens = df['cens']
tr = TobitModel()
tr = tr.fit(X, y, cens, verbose=False)
tr.coef_

array([ 294.44827939, -827.78090417, -140.01430074,  -24.97854148,
        103.69518044])

In [ ]:
df["whrs"] = df["whrs"].apply(lambda x: max(x, 0))
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]
X = sm.add_constant(X)
df['cens'] = np.where(df['whrs'] == 0, 0, 0)
cens = df['cens']
tr = TobitModel()
tr = tr.fit(X, y, cens, verbose=False)
tr.coef_

C:\Users\HP\AppData\Local\Temp\ipykernel_7068\1321454343.py:16: UserWarning: No censored observations; use regression methods for uncensored data
  warnings.warn("No censored observations; use regression methods for uncensored data")


array([ 470.02964199, -462.12329817,  -91.14099631,  -13.15770391,
         53.26155967])

In [ ]:
import numpy as np
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

class Tobit(GenericLikelihoodModel):
    def __init__(self, endog, exog, left=0, **kwds):
        self.left = left
        super().__init__(endog, exog, **kwds)

    def loglike(self, params):
        # Separar beta (coeficientes) y sigma (desviación estándar)
        beta = params[:-1]  # Los coeficientes
        sigma = params[-1]  # La desviación estándar (último parámetro)
        
        # Predicciones lineales
        xb = np.dot(self.exog, beta)
        y = self.endog

        # Calcular la log-verosimilitud
        ll = np.where(y > self.left,
                      np.log(1 / sigma) + sm.distributions.norm.logpdf((y - xb) / sigma),
                      sm.distributions.norm.logcdf((self.left - xb) / sigma))
        
        return ll.sum()

# Cargar datos
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]
y[y < 0] = 0  # Censuramos en 0

# Ajustar modelo
X = sm.add_constant(X)
model = Tobit(y, X, left=0)
result = model.fit()
print(result.summary())

C:\Users\HP\AppData\Local\Temp\ipykernel_21004\3479087987.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y < 0] = 0  # Censuramos en 0


ValueError: shapes (250,5) and (4,) not aligned: 5 (dim 1) != 4 (dim 0)

In [ ]:
from py4etrics.tobit import Tobit
import statsmodels.api as sm

y = df["whrs"]
x = df[["kl6", "k618", "wa", "we"]]
X = sm.add_constant(x)
df['cens'] = np.where(df['whrs'] == 0, -1, 0)
cens = df['cens']
model = Tobit(y, X, cens, left=0).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 5.468361
         Iterations: 566
         Function evaluations: 894


<class 'statsmodels.iolib.summary.Summary'>
"""
                              Tobit Regression Results                             
===================================================================================
Dep. Variable:                        whrs   Pseudo R-squ:                    0.008
Method:                 Maximum Likelihood   Log-Likelihood:                -1367.1
No. Observations:                      250   LL-Null:                       -1378.6
No. Uncensored Obs:                    150   LL-Ratio:                         23.0
No. Left-censored Obs:                 100   LLR p-value:                     0.000
No. Right-censored Obs:                  0   AIC:                            2744.2
Df Residuals:                          245   BIC:                            2761.8
Df Model:                                4   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        589.0003    841.589      0.700      0.484   -1060.483    2238.484
kl6         -827.7658    214.752     -3.855      0.000   -1248.672    -406.860
k618        -140.0192     74.227     -1.886      0.059    -285.502       5.463
wa           -24.9792     13.257     -1.884      0.060     -50.963       1.004
we           103.6896     41.826      2.479      0.013      21.712     185.668
Log(Sigma)     7.1777      0.063    113.628      0.000       7.054       7.302
==============================================================================
"""

In [ ]:
from py4etrics.truncreg import Truncreg
import statsmodels.api as sm

y = df["whrs"]
x = df[["kl6", "k618", "wa", "we"]]
X = sm.add_constant(x)
model = Truncreg(y, X, left=0).fit()
model.summary()

Optimization terminated successfully.
         Current function value: 7.658411
         Iterations: 4994
         Function evaluations: 7886


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\base\model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


<class 'statsmodels.iolib.summary.Summary'>
"""
                         Truncreg Regression Results                          
==============================================================================
Dep. Variable:                   whrs   Pseudo R-squ:                    0.003
Model:                       Truncreg   Log-Likelihood:                -1914.6
Method:            Maximum Likelihood   LL-Null:                       -1921.1
Date:                Sun, 23 Feb 2025   LL-Ratio:                         13.0
Time:                        00:16:22   LLR p-value:                     0.011
No. Observations:                 250   AIC:                            3839.2
Df Residuals:                     245   BIC:                            3856.8
Df Model:                           4   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.664e+10        nan        nan        nan         nan         nan
kl6        -5.197e+09        nan        nan        nan         nan         nan
k618       -1.261e+08        nan        nan        nan         nan         nan
wa         -8.649e+07        nan        nan        nan         nan         nan
we          1.056e+08        nan        nan        nan         nan         nan
Log(Sigma)    15.1967        nan        nan        nan         nan         nan
==============================================================================
"""

In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import Bounds, minimize
import re

def truncreg(formula, data, point, direction, scaled=False, iterlim=50):
    if direction not in ["left", "right"]:
        raise ValueError("'direction' must be 'left' or 'right'")
    
    x_vars = re.split(r'[*+]', re.search(r'~\s*(.+)', formula).group(1).strip())
    x_vars = [var.strip() for var in x_vars]
    x = data[[var for var in x_vars if var]]
    y = data[formula.split('~')[0].strip()]
    
    intercept = np.ones((x.shape[0], 1))
    x = np.hstack((intercept, x))
    
    if direction == "left" and any(y < point):
        raise ValueError("Data non-truncated, contains observations < '{}'".format(point))
    if direction == "right" and any(y > point):
        raise ValueError("Data non-truncated, contains observations > '{}'".format(point))

    def maxLikTruncreg(param, x, y, point, direction, scaled=scaled):
        epsilon = 1e-8
        sign = 1 if direction == "left" else -1
        beta = param[:-1]
        sigma = param[-1]
        bX = np.dot(x, beta)
        
        if not scaled:
            resid = y - bX
            trunc = bX - point
    
            #Potential Overflow Handling
            exp_argument = norm.logpdf(sign * trunc / (sigma + epsilon)) - norm.logcdf(sign * trunc / (sigma + epsilon))
            exp_argument = np.clip(exp_argument, a_min=None, a_max=700)
            mills = np.exp(exp_argument)
    
            lnL = -np.log(sigma + epsilon) + norm.logpdf(resid / (sigma + epsilon)) - norm.logcdf(sign * trunc / (sigma + epsilon))
            logLik = lnL.sum() if isinstance(lnL, np.ndarray) and lnL.ndim > 0 else lnL
    
            # Gradient calculation
            gbX = resid / (sigma ** 2 + epsilon) - sign * mills / (sigma + epsilon)
            gsigma = -1 / (sigma + epsilon) + resid ** 2 / (sigma + epsilon) ** 3 + sign * mills * trunc / (sigma + epsilon) ** 2
            gradient = np.concatenate([gbX, gsigma])
    
            # Hessian calculation
            bb = -1 / (sigma ** 2 + epsilon) + mills * (sign * trunc / (sigma + epsilon) + mills) / (sigma + epsilon) ** 2
            ss = 1 / (sigma + epsilon) ** 2 - 3 * resid ** 2 / (sigma + epsilon) ** 4 - 2 * mills * sign * trunc / (sigma + epsilon) ** 3 + \
                 mills * (sign * trunc / (sigma + epsilon) + mills) * trunc ** 2 / (sigma + epsilon) ** 4
            bs = -2 * resid / (sigma + epsilon) ** 3 + sign * mills / (sigma + epsilon) ** 2 - \
                 mills * (mills + sign * trunc / (sigma + epsilon)) * trunc / (sigma + epsilon) ** 3
            bs = np.array(bs)
    
            bb_matrix = np.dot(x.T, np.reshape(bb, (-1, 1)) * x)
            bs_vector = np.sum(np.reshape(bs, (-1, 1)) * x, axis=0)
            bs_vector = np.reshape(bs_vector, (-1, 1))
            ss_scalar = np.array([[np.sum(ss)]])
    
            hessian = np.block([[bb_matrix, bs_vector],
                            [bs_vector.T, ss_scalar]])
        else:
            exp_argument = norm.logpdf(sign * (bX - point / (sigma + epsilon))) - norm.logcdf(sign * (bX - point / (sigma + epsilon)))
            exp_argument = np.clip(exp_argument, a_min=None, a_max=700)
            mills = np.exp(exp_argument)
            
            lnL = -np.log(sigma + epsilon) + norm.logpdf(y / (sigma + epsilon) - bX) - norm.logcdf(sign * (bX - point / (sigma + epsilon)))
            logLik = lnL.sum() if isinstance(lnL, np.ndarray) and lnL.ndim > 0 else lnL
            
            #Gradient calculation
            gbX = (y / sigma - bX + epsilon) - mills * sign
            gsigma = -1 / (sigma + epsilon) + (y / sigma + epsilon - bX) * y /(sigma + epsilon) ** 2 - sign * mills * point/(sigma + epsilon) ** 2
            gradient = np.concatenate([gbX, gsigma])
            
            #Hessian Calculation
            bb = -1 + mills * (mills + sign * (bX - point / (sigma + epsilon)))
            bs = -y / sigma ** 2 + (mills + sign * (bX - point / (sigma + epsilon))) * mills * point / (sigma + epsilon) ** 2
            ss = 1 / (sigma + epsilon) ** 2 - 3 * y ** 2 / (sigma + epsilon) ** 4 + 2 * bX * y / (sigma + epsilon) ** 3 + \
                mills * (mills + sign * (bX - point / (sigma + epsilon))) * point ** 2 / (sigma + epsilon) ** 4 + \
                2 * sign * mills * point / (sigma + epsilon) ** 3
            bs = np.array(bs)
            
            bb_matrix = np.dot(x.T, np.reshape(bb, (-1, 1)) * x)
            bs_vector = np.sum(np.reshape(bs, (-1, 1)) * x, axis=0)
            bs_vector = np.reshape(bs_vector, (-1, 1))
            ss_scalar = np.array([[np.sum(ss)]])
            
            hessian = np.block([[bb_matrix, bs_vector],
                            [bs_vector.T, ss_scalar]])
            
        return {
            'logLik': logLik, 
            'gradient': gradient, 
            'hessian': hessian
        }

    def objective(param):
        result = maxLikTruncreg(param, x, y, point, direction)
        return -result['logLik']

    linmod = np.linalg.lstsq(x, y, rcond=None)
    start_beta = linmod[0]
    start_sigma = np.array([np.std(linmod[1])])
    start = np.concatenate([start_beta, start_sigma])

    lower_bounds = [np.NINF] * (len(start) - 1) + [1e-4]
    upper_bounds = [np.inf] * len(start)
    bounds = Bounds(lower_bounds, upper_bounds)

    result = minimize(objective, start, method='L-BFGS-B', bounds=bounds, options={'maxiter': iterlim})
    opt_result = maxLikTruncreg(result.x, x, y, point, direction, scaled)
    vcov = -np.linalg.inv(opt_result['hessian'])
    std_errors = np.sqrt(np.diag(vcov))
    
    return {
        'result': result,
        'vcov': vcov,
        'SE': std_errors
    }

In [ ]:
result = truncreg(formula='whrs~kl6+k618+wa+we', data=df, point=0, direction='left')
print(result)

{'result':   message: STOP: TOTAL NO. of ITERATIONS REACHED LIMIT
  success: False
   status: 1
      fun: 2108.38559242432
        x: [ 9.393e+02 -4.677e+02 -9.946e+01 -1.575e+01  5.119e+01
             5.322e+02]
      nit: 50
      jac: [-1.723e-02  3.402e-02  2.765e-02 -1.001e+00 -2.767e-01
            -9.420e-01]
     nfev: 455
     njev: 65
 hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>, 'vcov': array([[ 1.43858120e+05, -1.25420225e+04, -6.10188647e+03,
        -1.73430874e+03, -4.60015031e+03, -1.07771422e+02],
       [-1.25420225e+04,  1.14460624e+04,  4.09661983e+02,
         2.69847936e+02, -1.29942821e+02,  1.01826624e+02],
       [-6.10188647e+03,  4.09661983e+02,  1.26477413e+03,
         9.35012849e+01,  3.27066443e+01,  5.75536589e+00],
       [-1.73430874e+03,  2.69847936e+02,  9.35012849e+01,
         3.54375023e+01,  3.49471335e+00, -6.66373172e-01],
       [-4.60015031e+03, -1.29942821e+02,  3.27066443e+01,
         3.49471335e+00,  3.54336606e+02, -3.298341

In [ ]:
result

{'result':   message: STOP: TOTAL NO. of ITERATIONS REACHED LIMIT
   success: False
    status: 1
       fun: 2108.38559242432
         x: [ 9.393e+02 -4.677e+02 -9.946e+01 -1.575e+01  5.119e+01
              5.322e+02]
       nit: 50
       jac: [-1.723e-02  3.402e-02  2.765e-02 -1.001e+00 -2.767e-01
             -9.420e-01]
      nfev: 455
      njev: 65
  hess_inv: <6x6 LbfgsInvHessProduct with dtype=float64>,
 'vcov': array([[ 1.43858120e+05, -1.25420225e+04, -6.10188647e+03,
         -1.73430874e+03, -4.60015031e+03, -1.07771422e+02],
        [-1.25420225e+04,  1.14460624e+04,  4.09661983e+02,
          2.69847936e+02, -1.29942821e+02,  1.01826624e+02],
        [-6.10188647e+03,  4.09661983e+02,  1.26477413e+03,
          9.35012849e+01,  3.27066443e+01,  5.75536589e+00],
        [-1.73430874e+03,  2.69847936e+02,  9.35012849e+01,
          3.54375023e+01,  3.49471335e+00, -6.66373172e-01],
        [-4.60015031e+03, -1.29942821e+02,  3.27066443e+01,
          3.49471335e+00,  3.54

In [4]:
import copy
import logging

import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize
from scipy.special import log_ndtr
from scipy.stats import norm
from statsmodels.api import OLS
from statsmodels.regression.linear_model import (
    OLSResults, # noqa
    RegressionResultsWrapper,
)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Tobit(OLS):
    def __init__(
        self,
        endog,
        exog=None,
        reparam=True,
        c_lw=0.0,
        c_up=None,
        ols_option=True,
        missing="none",
        hasconst=None,
        **kwargs
    ):
        """

        :param endog: np.ndarray or pd.Series or pd.DataFrame - endogenous variable
        :param exog: np.ndarray or pd.Series - exogenous variable(s)
        :param reparam: bool - specifies whether to use Olsen's reparameterization
        :param c_lw: int or float or floating - lower censoring limit
        :param c_up: int or float or floating - upper censoring limit
        :param ols_option: bool - specifies whether to use OLS analytical
        solution or MLE when no threshold value is given
        :param missing: see OLS documentation
        :param hasconst: see OLS documentation
        :param kwargs: see OLS documentation
        """
        self._c_lw = c_lw
        self._c_up = c_up
        self.reparam = reparam
        self.scale = None
        self.params = None
        self.ols_params = None
        self.ols_option = ols_option

        endog_copy = copy.deepcopy(endog)
        exog_copy = copy.deepcopy(exog)

        super().__init__(endog_copy, exog_copy, missing=missing, hasconst=hasconst, **kwargs)
        if isinstance(endog, (pd.DataFrame, pd.Series)):
            endog = endog.values
        if isinstance(exog, pd.DataFrame):
            exog = exog.values
        if c_lw is not None and c_up is None:
            self.left_endog = endog[endog == c_lw]
            self.left_exog = exog[endog == c_lw, :]
            self.free_endog = endog[endog > c_lw]
            self.free_exog = exog[endog > c_lw, :]
        elif c_lw is None and c_up is not None:
            self.free_endog = endog[endog < c_up]
            self.free_exog = exog[endog < c_up, :]
            self.right_endog = endog[endog == c_up]
            self.right_exog = exog[endog == c_up, :]
        elif c_lw is not None and c_up is not None:
            self.left_endog = endog[endog == c_lw]
            self.left_exog = exog[endog == c_lw]
            self.free_endog = endog[(endog > c_lw) & (endog < c_up)]
            self.free_exog = exog[
                (endog > c_lw) & (endog < c_up), :
            ]
            self.right_endog = endog[endog == c_up]
            self.right_exog = exog[endog == c_up]
        else:
            self.free_endog = endog
            self.free_exog = exog
            
            logger.info(
                "No censoring threshold provided; OLS will be used for model estimation."
            )

    @property
    def c_lw(self):
        return self._c_lw

    @property
    def c_up(self):
        return self._c_up

    @c_lw.setter
    def c_lw(self, new_value):
        raise AttributeError("Cannot modify lower threshold value after it is set.")

    @c_up.setter
    def c_up(self, new_value):
        raise AttributeError("Cannot modify upper threshold value after it is set.")

    def neg_llh_jac(self, params):
        scale, betas = params[0], params[1:]

        scale_jac_censored = 0
        betas_jac_censored = np.zeros(len(betas))
        if self._c_lw is not None and self.left_endog.size:
            left_zscores = (self.left_endog - np.dot(self.left_exog, betas)) / scale
            left_d_dscale = np.dot(
                norm.pdf(left_zscores) / norm.cdf(left_zscores), -left_zscores / scale
            )
            left_d_dbetas = np.dot(
                norm.pdf(left_zscores) / norm.cdf(left_zscores), -self.left_exog / scale
            )
            scale_jac_censored += left_d_dscale
            betas_jac_censored += left_d_dbetas

        if self._c_up is not None and self.right_endog.size:
            right_zscores = (np.dot(self.right_exog, betas) - self.right_endog) / scale
            right_d_dscale = np.dot(
                norm.pdf(right_zscores) / norm.cdf(right_zscores),
                -right_zscores / scale,
            )
            right_d_dbetas = np.dot(
                norm.pdf(right_zscores) / norm.cdf(right_zscores),
                self.right_exog / scale,
            )
            scale_jac_censored += right_d_dscale
            betas_jac_censored += right_d_dbetas

        free_zscores = (self.free_endog - np.dot(self.free_exog, betas)) / scale
        free_d_dscale = np.sum(free_zscores**2 - 1) / scale
        free_d_dbetas = np.dot(free_zscores, self.free_exog / scale)

        scale_jac = scale_jac_censored + free_d_dscale
        betas_jac = betas_jac_censored + free_d_dbetas

        return -np.append(scale_jac, betas_jac)

    def loglike(self, params):  # noqa
        if hasattr(self, "llh"):
            return self.llh
        else:
            return super().loglike(params)

    def neg_llh_func(self, params):
        scale, betas = params[0], params[1:]

        llf_censored = 0

        if self._c_lw is not None and self.left_endog.size:
            llf_left = np.sum(
                log_ndtr((self.left_endog - np.dot(self.left_exog, betas)) / scale)
            )
            llf_censored += llf_left
        if self._c_up is not None and self.right_endog.size:
            llf_right = np.sum(
                log_ndtr((np.dot(self.right_exog, betas) - self.right_endog) / scale)
            )
            llf_censored += llf_right

        llf_free = np.sum(
            norm.logpdf((self.free_endog - np.dot(self.free_exog, betas)) / scale)
            - np.log(max(scale, np.finfo("float").resolution))
        )

        return -1 * (llf_censored + llf_free)

    # Reparameterization
    def neg_llh_func2(self, params):
        gamma, delta = params[0], params[1:]

        llf_censored = 0
        if self._c_lw is not None and self.left_endog.size:
            llf_left = np.sum(
                log_ndtr(gamma * self.left_endog - np.dot(self.left_exog, delta))
            )
            llf_censored += llf_left
        if self._c_up is not None and self.right_endog.size:
            llf_right = np.sum(
                log_ndtr(np.dot(self.right_exog, delta) - gamma * self.right_endog)
            )
            llf_censored += llf_right

        llf_free = np.sum(
            np.log(max(gamma, np.finfo("float").resolution))
            + norm.logpdf(gamma * self.free_endog - np.dot(self.free_exog, delta))
        )

        return -1 * (llf_censored + llf_free)

    def neg_llh_jac2(self, params):
        gamma, delta = params[0], params[1:]

        gamma_jac_censored = 0
        delta_jac_censored = np.zeros(len(delta))

        if self._c_lw is not None and self.left_endog.size:
            left_zscore = gamma * self.left_endog - np.dot(self.left_exog, delta)
            left_d_dgamma = np.dot(
                norm.pdf(left_zscore) / norm.cdf(left_zscore), self.left_endog
            )
            left_d_ddelta = np.dot(
                norm.pdf(left_zscore) / norm.cdf(left_zscore), -self.left_exog
            )
            delta_jac_censored += left_d_ddelta
            gamma_jac_censored += left_d_dgamma

        if self._c_up is not None and self.right_endog.size:
            right_zscore = np.dot(self.right_exog, delta) - gamma * self.right_endog
            right_d_dgamma = np.dot(
                norm.pdf(right_zscore) / norm.cdf(right_zscore), -self.right_endog
            )
            right_d_ddelta = np.dot(
                norm.pdf(right_zscore) / norm.cdf(right_zscore), self.right_exog
            )
            delta_jac_censored += right_d_ddelta
            gamma_jac_censored += right_d_dgamma

        free_zscore = gamma * self.free_endog - np.dot(self.free_exog, delta)
        free_d_dgamma = np.sum(1 / gamma - free_zscore * self.free_endog)
        free_d_ddelta = np.dot(free_zscore, self.free_exog)

        gamma_jac = gamma_jac_censored + free_d_dgamma
        delta_jac = delta_jac_censored + free_d_ddelta

        return -np.append(gamma_jac, delta_jac)

    def fit_tobit(self, cov_type="HC1", cov_kwds=None, use_t=None, verbose=True):
        modl = super().fit(cov_type=cov_type)  # noqa
        self.ols_params = copy.deepcopy(modl.params)
        self.scale = np.sqrt(np.cov(modl.resid))  # noqa

        self.run_optimize(verbose)

        lfit = OLSResults(
            self,
            self.params,
            normalized_cov_params=self.normalized_cov_params,
            scale=self.scale,
            cov_type=cov_type,
            cov_kwds=cov_kwds,
            use_t=use_t,
        )

        return RegressionResultsWrapper(lfit)

    def fit(self, cov_type="HC1", cov_kwds=None, use_t=None, verbose=False, **kwargs):
        if (
            self._c_lw is None
            and self._c_up is None
            and self.ols_option
        ):
            return super().fit(cov_type=cov_type, cov_kwds=None, use_t=None, **kwargs)
        else:
            return self.fit_tobit(
                cov_type=cov_type, cov_kwds=None, use_t=None, verbose=verbose
            )

    def run_optimize(self, verbose):
        initial_params, func, jac = self.get_initial_params_and_functions()
        result = minimize(
            func,
            initial_params,
            method="BFGS",
            jac=jac,
            options={"disp": verbose},
        )
        self.update_model_parameters(result)
        if verbose:
            logger.info(result)

    def get_initial_params_and_functions(self):
        if self.reparam:
            initial_params = np.append(1 / self.scale, self.ols_params / self.scale)
            func = self.neg_llh_func2
            jac = self.neg_llh_jac2
        else:
            initial_params = np.append(self.scale, self.ols_params)
            func = self.neg_llh_func
            jac = self.neg_llh_jac
        return initial_params, func, jac

    def update_model_parameters(self, result):
        self.llh = -result.fun  # noqa
        if self.reparam:
            self.scale = 1 / result.x[0]
            self.params = result.x[1:] * self.scale
        else:
            self.scale = result.x[0]
            self.params = result.x[1:]
        self.normalized_cov_params = result.hess_inv[1:, 1:]

In [7]:
import statsmodels.api as sm
y = df["whrs"]
X = df[["kl6", "k618", "wa", "we"]]
X = sm.add_constant(X)
model = Tobit(df["whrs"], df[["kl6", "k618", "wa", "we"]], c_lw=0)
model.fit(verbose=True)

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_minimize.py:726: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)
INFO:__main__:  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 1367.333919287908
        x: [ 7.606e-04 -5.937e-01 -8.880e-02 -1.361e-02  9.326e-02]
      nit: 13
      jac: [ 3.048e-04  8.574e-07  1.390e-06 -7.101e-06  1.386e-06]
 hess_inv: [[ 2.300e-09 -8.432e-07 ...  3.424e-09  1.817e-07]
            [-8.432e-07  2.384e-02 ...  3.278e-04 -1.528e-03]
            ...
            [ 3.424e-09  3.278e-04 ...  4.090e-05 -1.468e-04]
            [ 1.817e-07 -1.528e-03 ... -1.468e-04  6.009e-04]]
     nfev: 20
     njev: 20


         Current function value: 1367.333919
         Iterations: 13
         Function evaluations: 20
         Gradient evaluations: 20


In [8]:
import numpy as np
import logging
import enum
from scipy.stats import norm
from typing import Tuple


@enum.unique
class TobitType(enum.IntEnum):
    """ Type of Tobit Regression Model """
    TYPE1 = 1,
    TYPE2 = 2,
    TYPE3 = 3


class RegressionResult(object):
    """ Regression results class """
    def __init__(self):
        self._history = []
        self._params = None
        self._vars = None
        self._fittedModel = None

    @property
    def history(self):
        return self._history

    @property
    def parameters(self):
        return self._params

    @parameters.setter
    def parameters(self, value):
        self._params = value
        if len(value) == 2:
            self._params = value[0]

    @property
    def varnames(self):
        return self._vars

    @varnames.setter
    def varnames(self, value):
        self._vars = value

    def setModel(self, fitted_model):
        self._fittedModel = fitted_model

    def appendDiff(self, diff):
        """
        Append parameter diff across iterations to history
        :param diff: mean sum of square diff in model parameters across iterations
        """
        self._history.append(diff)

    def predict(self, exog: np.ndarray) -> np.ndarray:
        """
        Predict the model output. Model must have been fitted to data
        :param exog: exogeneous variables (X)
        :return: model output (y)
        """
        return self._fittedModel._predict(exog, self._params)


class TobitRegression(object):
    """ Tobit (censored) regression model """
    DEFAULT_DTYPE = np.float32

    def __init__(self, low=None, high=None, type=TobitType.TYPE1, diff_thresh=1E-10, niters=1000, dtype=np.float32):
        """
        Initialize the regression model
        :param low: Low threshold for censoring
        :param high: High threshold for censoring
        :param type: Tobit type 1, 2 or 3. Currently only type 1 is supported. Future releases will add support for
        type 2 and type 3 Tobit regressions
        :param diff_thresh: Stop iterations when diff between successive iterations becomes lower than this threshold
        :param niters: Maximum number of iterations
        :param dtype: data type of numeric variables. Default is 32 bit float
        """
        assert (low is not None) or (high is not None), "both low and high cannot be None"
        if (low is not None) and (high is not None):
            assert low < high, "low must be strictly less than high"
        self.DEFAULT_DTYPE = dtype
        self.diffThreshold = diff_thresh
        self.maxIters = niters
        self.delta = None
        self.gamma = None
        self.low = low
        self.high = high
        self.aIndex = np.array([])
        self.bIndex = np.array([])
        self.midIndex = np.array([])
        self.funcVal = None
        self.funcDeriv = None
        self.categoricalMap = {}
        self.colNames = []
        self.categoricalColInd = None
        self.hasConst = None
        self.logger = logging.getLogger(self.__class__.__name__)

    @property
    def parameters(self):
        return self.delta/self.gamma, 1.0/self.gamma

    def initializeParams(self, nexog: int, endog: np.ndarray) -> None:
        """
        Initialize model parameters
        :param nexog: number of exogeneous (independent) variables
        :param endog: endogeneous (dependent) variable array
        """
        self.delta = np.ones(nexog, dtype=self.DEFAULT_DTYPE)
        self.funcVal = np.zeros(nexog + 1, dtype=self.DEFAULT_DTYPE)
        self.funcDeriv = np.zeros((nexog + 1, nexog + 1), dtype=self.DEFAULT_DTYPE)
        self.gamma = 1.0
        indicA = 0
        indicB = 0
        if self.low is not None:
            indicA = np.less_equal(endog, self.low)
            if indicA.sum():
                self.aIndex = np.where(indicA)[0]
        if self.high is not None:
            indicB = np.greater_equal(endog, self.high)
            if indicB.sum():
                self.bIndex = np.where(indicB)[0]
        self.midIndex = np.where(1 - indicA - indicB)[0]

    def lowLikelihood(self, exog: np.ndarray) -> float:
        """
        Calculate log likelihood of output value at lower threshold
        :param exog: Array of exogeneous variables
        :return: Log likelihood of output at lower threshold
        """
        required = self.aIndex
        xdelta = self.low * self.gamma - np.einsum("ij,j->i", exog[required, :], self.delta)
        cdf = norm.cdf(xdelta)
        return np.log(cdf).sum()

    def highLikelihood(self, exog: np.ndarray) -> float:
        """
        Calculate log likelihood of output value at higher threshold
        :param exog: Array of exogeneous variables
        :return: Log likelihood of output value at lower threshold
        """
        required = self.bIndex
        xdelta = -self.high * self.gamma + np.einsum("ij,j->i", exog[required, :], self.delta)
        cdf = norm.cdf(xdelta)
        return np.log(cdf).sum()

    def midLikelihood(self, exog: np.ndarray, endog: np.ndarray) -> float:
        """
        Calculate log likelihood of output value not hitting lower or higher thresholds (in uncensored region)
        :param exog: Array of exogeneous variables
        :param endog: Array of endogeneous (output) variables
        :return: log likelihood of output value in uncensored region
        """
        required = self.midIndex
        xdelta = self.gamma * endog[required] - np.einsum("ij,j->i", exog[required, :], self.delta)
        pdf = norm.pdf(xdelta)
        return np.log(pdf * self.gamma).sum()

    def logLikelihood(self, exog: np.ndarray, endog: np.ndarray) -> float:
        """
        Log likelihood of data subject to lower and higher thresholds (censoring)
        :param exog: Array of exogeneous variables
        :param endog: Array of endogeneous variables
        :return: Log likelihood of data subject to lower and higher thresholds
        """
        result = 0
        if self.low is not None:
            result += self.lowLikelihood(exog)
        if self.high is not None:
            result += self.highLikelihood(exog)

        result += self.midLikelihood(exog, endog)
        return result

    def lowFunc(self, exog: np.ndarray, val: np.ndarray) -> None:
        """
        Function (= derivative of log likelihood) at lower censoring threshold
        :param exog: array of exogeneous variables
        :param val: function value at lower threshold
        """
        required = self.aIndex
        xdelta = self.low * self.gamma - np.einsum("ij,j->i", exog[required, :], self.delta)
        den = norm.cdf(xdelta)
        num = norm.pdf(xdelta)
        limindex = (den == 0.) & (num == 0.)
        den = np.where(limindex, 1.0, den)
        num = np.where(limindex, 1.0, num)
        frac = num / den
        val[0:-1] += -np.einsum("i,ij->j", frac, exog[required, :])
        val[-1] += frac.sum() * self.low

    def highFunc(self, exog: np.ndarray, val: np.ndarray) -> None:
        """
        Function at higher censoring threshold
        :param exog: array of exogeneous variables
        :param val: value of function at upper censoring threshold
        """
        required = self.bIndex
        xdelta = -self.high * self.gamma + np.einsum("ij,j->i", exog[required, :], self.delta)
        den = norm.cdf(xdelta)
        num = norm.pdf(xdelta)
        limindex = (den == 0.) & (num == 0.)
        den = np.where(limindex, 1.0, den)
        num = np.where(limindex, 1.0, num)
        frac = num / den
        val[0:-1] += np.einsum("i,ij->j", frac, exog[required, :])
        val[-1] += -frac.sum() * self.high

    def midFunc(self, exog: np.ndarray, endog: np.ndarray, val: np.ndarray) -> None:
        """
        Function value in uncensored region
        :param exog: array of exogeneous variables
        :param endog: array of output variables
        :param val: value of function
        """
        required = self.midIndex
        xdelta = self.gamma * endog[required] - np.einsum("ij,j->i", exog[required, :], self.delta)
        val[0:-1] += np.einsum("i,ij->j", xdelta, exog[required, :])
        val[-1] += 1.0 / self.gamma - np.multiply(xdelta, endog[required]).sum()

    def function(self, exog: np.ndarray, endog: np.ndarray) -> np.ndarray:
        """
        Function (derivative of log likelihood) that we are trying to determine a root of
        :param exog: array of exogeneous variables (X)
        :param endog: array of output variables (y)
        :return: array of function values evaluated at specified X and y
        """
        val = self.funcVal
        val[:] = 0
        if self.aIndex.shape[0]:
            self.lowFunc(exog, val)
        if self.bIndex.shape[0]:
            self.highFunc(exog, val)
        self.midFunc(exog, endog, val)
        return val

    def lowDeriv(self, exog: np.ndarray, val: np.ndarray) -> None:
        """
        Derivative of function (which was derivative of log likelihood) at lower censoring threshold
        :param exog: array of exogeneous variables
        :param val: Derivative of function (2 dimensional array) at lower censoring threshold
        """
        required = self.aIndex
        xdelta = self.low * self.gamma - np.einsum("ij,j->i", exog[required, :], self.delta)
        den = norm.cdf(xdelta)
        num = norm.pdf(xdelta)
        limindex = (den == 0.) & (num == 0.)
        den = np.where(limindex, 1.0, den)
        num = np.where(limindex, 1.0, num)
        frac = num / den
        termfrac = np.multiply(frac, 1 - frac)
        prod = np.multiply(frac, xdelta)
        termprod = np.multiply(prod, 1 - prod)
        val[0:-1, 0:-1] += np.einsum("i,ij,ik->jk", termprod, exog[required, :], exog[required, :])
        val[-1, -1] += termfrac.sum() * self.low * self.low
        crossDeriv = -self.low * np.einsum("i,ij->j", termfrac, exog[required, :])
        val[-1, 0:-1] += crossDeriv
        val[0:-1, -1] += crossDeriv

    def highDeriv(self, exog: np.ndarray, val: np.ndarray) -> None:
        """
         Derivative of function (which was derivative of log likelihood) at upper censoring threshold
        :param exog: array of exogeneous variables
        :param val: Derivative of function (2 dimensional array) at uppper censoring threshold
        """
        required = self.bIndex
        xdelta = -self.high * self.gamma + np.einsum("ij,j->i", exog[required, :], self.delta)
        den = norm.cdf(xdelta)
        num = norm.pdf(xdelta)
        limindex = (den == 0.) & (num == 0.)
        den = np.where(limindex, 1.0, den)
        num = np.where(limindex, 1.0, num)
        frac = num / den
        termfrac = np.multiply(frac, 1 - frac)
        prod = np.multiply(frac, xdelta)
        termprod = np.multiply(prod, 1 - prod)
        val[0:-1, 0:-1] += np.einsum("i,ij,ik->jk", termprod, exog[required, :], exog[required, :])
        val[-1, -1] += termfrac.sum() * self.high * self.high
        crossDeriv = -self.high * np.einsum("i,ij->j", termfrac, exog[required, :])
        val[-1, 0:-1] += crossDeriv
        val[0:-1, -1] += crossDeriv

    def midDeriv(self, exog: np.ndarray, endog: np.ndarray, val: np.ndarray) -> None:
        """
        Derivative of function (which was derivative of log likelihood) in uncensored region
        :param exog: array of exogeneous variables
        :param endog: array of output variables
        :param val: Derivative of function (2 dimensional array) in uncensoring region
        """
        required = self.midIndex
        val[0:-1, 0:-1] += -np.einsum("ij,ik->jk", exog[required, :], exog[required, :])
        val[-1, -1] += -(1.0 / (self.gamma * self.gamma) +
                         np.multiply(endog[required], endog[required]).sum())
        yixij = np.einsum("i,ij->j", endog[required], exog[required, :])
        val[-1, 0:-1] += yixij
        val[0:-1, -1] += yixij

    def derivative(self, exog: np.ndarray, endog: np.ndarray) -> np.ndarray:
        """
        Derivative of function (which was derivative of log likelihood function)
        :param exog: array of exogeneous variables
        :param endog: array of output variables
        :return: Derivative (2 dimensional array) of function. Function is the derivative of log likelihood
        """
        deriv = self.funcDeriv
        deriv[:, :] = 0
        if self.aIndex.shape[0]:
            self.lowDeriv(exog, deriv)
        if self.bIndex.shape[0]:
            self.highDeriv(exog, deriv)
        self.midDeriv(exog, endog, deriv)
        return deriv

    def newtonStep(self, exog: np.ndarray, endog: np.ndarray) -> float:
        """
        Perform a Newton step to update model parameters. \delta x = - f(x) / gradient(f(x))
        :param exog: array of exogeneous variables
        :param endog: array of output variables
        :return: diff or the change in parameters
        """
        fx = self.function(exog, endog)
        jacobian = self.derivative(exog, endog)
        step = np.linalg.solve(jacobian, -fx)
        self.delta += step[0:-1]
        self.gamma += step[-1]
        self.gamma = abs(self.gamma)
        return np.multiply(step, step).sum() / step.shape[0]

    def coerceData(self, exog: np.ndarray, endog: np.ndarray, categorical: tuple,
                   col_names: tuple) -> Tuple[np.ndarray, np.ndarray]:
        """
        Coerce data to default datatype
        :param exog:
        :param endog:
        :param categorical:
        :param col_names:
        :return:
        """
        self.categoricalMap = {}
        catset = set(categorical)
        noncat = [i for i in range(exog.shape[1]) if i not in catset]
        colnames = [col_names[i] for i in range(exog.shape[1]) if i not in catset]
        concatarr = [exog[:, noncat]]
        for cat in categorical:
            distinct = sorted(list(set(exog[:, cat])))
            self.categoricalMap[cat] = distinct
            catcols = np.zeros((exog.shape[0], len(distinct)-1), dtype=self.DEFAULT_DTYPE)
            for i, d in enumerate(distinct[0:-1]):
                catcols[:, i] = np.where(exog[:, cat] == d, 1, 0)
            concatarr.append(catcols)
            colnames.extend(["%s_%s" % (col_names[cat], si) for si in distinct[0:-1]])
        modExog = np.concatenate(concatarr, axis=1)
        self.colNames = colnames
        return modExog.astype(self.DEFAULT_DTYPE), endog.astype(self.DEFAULT_DTYPE)

    def fit(self, exog: np.ndarray, endog: np.ndarray, include_constant: bool = True, categorical: tuple = (),
            col_names: tuple = ()) -> RegressionResult:
        """
        Fir the Tobit regression model to the data
        :param exog: array of exogeneous variables
        :param endog: array of output variables
        :param include_constant: Add constant to exogeneous variables array
        :param categorical: tuple containing indices of categorical columns
        :param col_names: Column names (for ease of identification of fitted parameters)
        :return: Regression results
        """
        if not col_names:
            col_names = tuple("col_%d" % i for i in range(exog.shape[1]))
        self.hasConst = include_constant
        if include_constant:
            const = np.ones((exog.shape[0], 1), dtype=self.DEFAULT_DTYPE)
            exog = np.concatenate((exog, const), axis=1)
            col_names = col_names + ("intercept",)
        self.categoricalColInd = categorical
        exog, endog = self.coerceData(exog, endog, categorical, col_names)
        self.initializeParams(exog.shape[1], endog)
        diff = self.diffThreshold + 1
        iters = 0
        res = RegressionResult()
        while (diff > self.diffThreshold) and (iters < self.maxIters):
            diff = self.newtonStep(exog, endog)
            iters += 1
            res.appendDiff(diff)
        self.logger.info("Iterations: %d, final residual: %f", iters, diff)
        res.parameters = self.parameters
        res.varnames = self.colNames
        return res

    def _coerceExogForPred(self, exog: np.ndarray) -> np.ndarray:
        """
        Coerce exogeneous variables array during prediction to handle categorical variables
        :param exog: array of exogeneous variables
        :return: Processed exogeneous variables with categorical columns converted to indicator variable columns
        """
        catset = set(self.categoricalColInd)
        noncat = [i for i in range(exog.shape[1]) if i not in catset]
        concatarr = [exog[:, noncat]]
        for cat in self.categoricalColInd:
            distinct = sorted(list(set(exog[:, cat])))
            catcols = np.zeros((exog.shape[0], len(distinct) - 1), dtype=self.DEFAULT_DTYPE)
            for i, d in enumerate(distinct[0:-1]):
                catcols[:, i] = np.where(exog[:, cat] == d, 1, 0)
            concatarr.append(catcols)
        modExog = np.concatenate(concatarr, axis=1)
        return modExog.astype(self.DEFAULT_DTYPE)

    def _predict(self, exog: np.ndarray, params: np.ndarray) -> np.ndarray:
        """
        Predict the output of the model using exogeneous variables as input. This method should not be called directly.
        Use predict method from RegressionResults object
        :param exog: exogeneous variables (X)
        :param params: fitted model parameters (provided by RegressionResult object)
        :return: output value from the model (y)
        """
        if self.hasConst:
            exog2 = np.ndarray((exog.shape[0], exog.shape[1] + 1), dtype=self.DEFAULT_DTYPE)
            exog2[:, 0:-1] = exog
            exog2[:, -1] = 1.0
            exog = exog2

        exog = self._coerceExogForPred(exog)
        vals = np.einsum("ij,j->i", exog, params)
        if self.low:
            vals = np.where(vals <= self.low, self.low, vals)
        if self.high:
            vals = np.where(vals > self.high, self.high, vals)
        return vals

<>:317: SyntaxWarning: invalid escape sequence '\d'
<>:317: SyntaxWarning: invalid escape sequence '\d'
C:\Users\HP\AppData\Local\Temp\ipykernel_20364\1168873673.py:317: SyntaxWarning: invalid escape sequence '\d'
  """


In [9]:
model = TobitRegression(low=0)
result = model.fit(X.values, y.values)
print(result.parameters)

C:\Users\HP\AppData\Local\Temp\ipykernel_20364\1168873673.py:317: SyntaxWarning: invalid escape sequence '\d'
  """


LinAlgError: Singular matrix

In [1]:
import pandas as pd
import numpy as np
import pyreadstat as st
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2024 3T\[MT06] Microeconometría I\Clases\Stata\MROZ1.dta"

df, meta = st.read_dta(path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   lfp               753 non-null    int64  
 1   whrs              753 non-null    int64  
 2   kl6               753 non-null    int64  
 3   k618              753 non-null    int64  
 4   wa                753 non-null    int64  
 5   we                753 non-null    int64  
 6   ww                753 non-null    float64
 7   rpwg              753 non-null    float64
 8   hhrs              753 non-null    int64  
 9   ha                753 non-null    int64  
 10  he                753 non-null    int64  
 11  hw                753 non-null    float64
 12  faminc            753 non-null    int64  
 13  mtr               753 non-null    float64
 14  wmed              753 non-null    int64  
 15  wfed              753 non-null    int64  
 16  un                753 non-null    float64
 1

In [7]:
import statsmodels.api as sm
x = df['salario_ofrecido']  #Variable independiente sin constante.
X = sm.add_constant(x)
y = df['lfp']  #Variable dependiente.

# Ajustar el modelo logístico
probit_model = sm.Probit(y, X)
result = probit_model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.657366
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                    lfp   No. Observations:                  753
Model:                         Probit   Df Residuals:                      751
Method:                           MLE   Df Model:                            1
Date:                Sun, 23 Feb 2025   Pseudo R-squ.:                 0.03860
Time:                        17:14:57   Log-Likelihood:                -495.00
converged:                       True   LL-Null:                       -514.87
Covariance Type:            nonrobust   LLR p-value:                 2.882e-10
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.8502      0.172     -4.948      0.000      -1.187      -0.513
salario_ofrecido     0.2609      0.042      6.156      0.000       0.178       0.344
====================================================================================
"""

In [4]:
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel


class MyProbit(GenericLikelihoodModel):
    def loglike(self, params):
        exog = self.exog
        endog = self.endog
        q = 2 * endog - 1
        return stats.norm.logcdf(q*np.dot(exog, params)).sum()

In [5]:
sm_probit_manual = MyProbit(y, X).fit()
print(sm_probit_manual.summary())

Optimization terminated successfully.
         Current function value: 0.657366
         Iterations: 47
         Function evaluations: 89
                               MyProbit Results                               
Dep. Variable:                    lfp   Log-Likelihood:                -495.00
Model:                       MyProbit   AIC:                             994.0
Method:            Maximum Likelihood   BIC:                             1003.
Date:                Sun, 23 Feb 2025                                         
Time:                        17:13:18                                         
No. Observations:                 753                                         
Df Residuals:                     751                                         
Df Model:                           1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------

In [8]:
# Ajustar el modelo logístico
logit_model = sm.Logit(y, X)
result = logit_model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.657445
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    lfp   No. Observations:                  753
Model:                          Logit   Df Residuals:                      751
Method:                           MLE   Df Model:                            1
Date:                Sun, 23 Feb 2025   Pseudo R-squ.:                 0.03849
Time:                        17:15:12   Log-Likelihood:                -495.06
converged:                       True   LL-Null:                       -514.87
Covariance Type:            nonrobust   LLR p-value:                 3.062e-10
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -1.3784      0.284     -4.853      0.000      -1.935      -0.822
salario_ofrecido     0.4225      0.071      5.986      0.000       0.284       0.561
====================================================================================
"""

In [9]:
from statsmodels.base.model import GenericLikelihoodModel
import numpy as np
from scipy import stats

class MyLogit(GenericLikelihoodModel):
    def loglike(self, params):
        exog = self.exog
        endog = self.endog
        # Calcula el logit, que es el logaritmo de la probabilidad de éxito / la probabilidad de fracaso
        linear_pred = np.dot(exog, params)
        # Función de enlace logit: log(p / (1 - p))
        p = 1 / (1 + np.exp(-linear_pred))
        # Log-verosimilitud usando la distribución Bernoulli
        return (endog * np.log(p) + (1 - endog) * np.log(1 - p)).sum()
    
sm_logit_manual = MyLogit(y, X).fit()
print(sm_logit_manual.summary())

Optimization terminated successfully.
         Current function value: 0.657445
         Iterations: 49
         Function evaluations: 92
                               MyLogit Results                                
Dep. Variable:                    lfp   Log-Likelihood:                -495.06
Model:                        MyLogit   AIC:                             994.1
Method:            Maximum Likelihood   BIC:                             1003.
Date:                Sun, 23 Feb 2025                                         
Time:                        17:15:28                                         
No. Observations:                 753                                         
Df Residuals:                     751                                         
Df Model:                           1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------